In [47]:
import tensorflow as tf
import numpy as np

In [48]:
sample=" Mulmi team, cheer-up"
#" Mulmi team, cheer-u" -> "Mulmi team, cheer-up"

In [49]:
idx2char=list(set(sample))
char2idx={c:i for i, c in enumerate(idx2char)}

In [50]:
dic_size=len(char2idx)
hidden_size=num_classes=dic_size
#dic_size:원핫인코딩크기(rnn input size)
#hidden_size: output size
#num_classes:최종 output size(softmax)
batch_size=1 #데이터 1개 -> 배치 -> 업데이트
sequence_length=len(sample)-1
learning_rate=0.1

In [73]:
tf.reset_default_graph()
sample_idx=[char2idx[c] for c in sample]
#sample=" Mulmi team, cheer-up"
xdata=[sample_idx[:-1]]
ydata=[sample_idx[1:]]
x=tf.placeholder(tf.int32, [None,sequence_length])
y=tf.placeholder(tf.int32, [None,sequence_length])
x_one_hot=tf.one_hot(x,num_classes)
cell=tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
ini_state=cell.zero_state(batch_size, tf.float32)
outputs, _states=tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=ini_state, dtype=tf.float32)

In [74]:
xfc=tf.reshape(outputs, [-1, hidden_size])
outputs=tf.contrib.layers.fully_connected(xfc, num_classes, activation_fn=None)
outputs=tf.reshape(outputs,[batch_size,sequence_length, num_classes])
weights=tf.ones([batch_size, sequence_length])
sequence_loss=tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=y, weights=weights)
loss=tf.reduce_mean(sequence_loss)
train=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction=tf.argmax(outputs, axis=2)

In [75]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        cv, _=sess.run([loss, train], feed_dict={x:xdata, y:ydata})
        res=sess.run(prediction, feed_dict={x:xdata})
        res_str=[idx2char[c] for c in np.squeeze(res)]
        print(i,"loss:", cv, "prediction:", "".join(res_str))
        

0 loss: 2.7076817 prediction: eeu   e  e  uuuuuuuu
1 loss: 2.5466664 prediction: eemm         eeeeeee
2 loss: 2.3200142 prediction: emmmm meem  eeeeeuuu
3 loss: 2.0346828 prediction: emmmmmmmmm    ee   p
4 loss: 1.9836344 prediction: eulmm teem  eeeeuuup
5 loss: 1.5843931 prediction: eulm  teee, eeeeuupp
6 loss: 1.5017675 prediction: eulm  teem  cheer-pp
7 loss: 1.2140453 prediction: eulmi teem  cheer-up
8 loss: 1.008054 prediction: Mulmi teem, cheer-up
9 loss: 0.83967686 prediction: Mulmi teem, cheer-up
10 loss: 0.6757988 prediction: Mulmi teem, cheer-up
11 loss: 0.526719 prediction: Mulmi teem, cheer-up
12 loss: 0.40761495 prediction: Mulmi team, cheer-up
13 loss: 0.311414 prediction: Mulmi team, cheer-up
14 loss: 0.23282394 prediction: Mulmi team, cheer-up
15 loss: 0.17259482 prediction: Mulmi team, cheer-up
16 loss: 0.12826566 prediction: Mulmi team, cheer-up
17 loss: 0.094826624 prediction: Mulmi team, cheer-up
18 loss: 0.0709387 prediction: Mulmi team, cheer-up
19 loss: 0.0545869

In [76]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

dataset=[['사과','치즈','생수'],
         ['생수','호두','치즈','고등어'],
         ['수박','사과','생수'],
         ['생수','호두','치즈','옥수수']]

In [77]:
te=TransactionEncoder()
te_ary=te.fit(dataset).transform(dataset)
te_ary

array([[False,  True,  True, False, False,  True, False],
       [ True, False,  True, False, False,  True,  True],
       [False,  True,  True,  True, False, False, False],
       [False, False,  True, False,  True,  True,  True]])

In [78]:
df=pd.DataFrame(te_ary, columns=te.columns_)
df
freq_item=apriori(df, min_support=0.5, use_colnames=True)
freq_item

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(생수, 사과)"
5,0.75,"(생수, 치즈)"
6,0.50,"(생수, 호두)"
7,0.50,"(치즈, 호두)"
8,0.50,"(생수, 치즈, 호두)"


In [72]:
from mlxtend.frequent_patterns import association_rules
association_rules(freq_item, metric="lift", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
1,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
2,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0
3,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf
4,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
5,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
6,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
7,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
8,"(생수, 치즈)",(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
9,"(생수, 호두)",(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
